
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>



# Aggregation

##### Objectives
1. Group data by specified columns
1. Apply grouped data methods to aggregate data
1. Apply built-in functions to aggregate data

##### Methods
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html" target="_blank">DataFrame</a>: **`groupBy`**
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/grouping.html" target="_blank" target="_blank">Grouped Data</a>: **`agg`**, **`avg`**, **`count`**, **`max`**, **`sum`**
- <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html" target="_blank">Built-In Functions</a>: **`approx_count_distinct`**, **`avg`**, **`sum`**

In [0]:
%run ../Includes/Classroom-Setup

Python interpreter will be restarted.
Python interpreter will be restarted.


Resetting the learning environment:
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03"

Validating the locally installed datasets:
| listing local files...(3 seconds)
| validation completed...(3 seconds total)

Creating & using the schema "muaazkhurshid_q1yg_da_asp" in the catalog "spark_catalog"...(1 seconds)

Predefined tables in "muaazkhurshid_q1yg_da_asp":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/muaazkhurshid@uni-koblenz.de/apache-spark-programming-with-databricks
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/muaazkhurshid@uni-koblenz.de/apache-spark-programming-with-databricks/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/apache-spark-programming-with-databricks/v03
| DA.paths.checkpoints: dbfs:/mnt/dbacademy-users/muaazkhurshid@uni-koblenz.de/apache-spark-programming-with-databricks/_checkpoints

Setup completed (6 seconds)

P




Let's use the BedBricks events dataset.

In [0]:
df = spark.read.format("delta").load(DA.paths.events)
# display(df)




### Grouping data

<img src="https://files.training.databricks.com/images/aspwd/aggregation_groupby.png" width="60%" />




### groupBy
Use the DataFrame **`groupBy`** method to create a grouped data object. 

This grouped data object is called **`RelationalGroupedDataset`** in Scala and **`GroupedData`** in Python.

In [0]:
df.groupBy("event_name")

Out[5]: <pyspark.sql.group.GroupedData at 0x7fd11ad13c70>

In [0]:
df.groupBy("geo.state", "geo.city")

Out[6]: <pyspark.sql.group.GroupedData at 0x7fd11ad6d4f0>




### Grouped data methods
Various aggregation methods are available on the <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/grouping.html" target="_blank">GroupedData</a> object.


| Method | Description |
| --- | --- |
| agg | Compute aggregates by specifying a series of aggregate columns |
| avg | Compute the mean value for each numeric columns for each group |
| count | Count the number of rows for each group |
| max | Compute the max value for each numeric columns for each group |
| mean | Compute the average value for each numeric columns for each group |
| min | Compute the min value for each numeric column for each group |
| pivot | Pivots a column of the current DataFrame and performs the specified aggregation |
| sum | Compute the sum for each numeric columns for each group |

In [0]:
event_counts_df = df.groupBy("event_name").count()
display(event_counts_df)

event_name,count
mattresses,1197680
down,84004
press,224284
shipping_info,291802
main,2987366
warranty,253170
finalize,180678
login,34329
faq,279616
careers,68938





Here, we're getting the average purchase revenue for each.

In [0]:
avg_state_purchases_df = df.groupBy("geo.state").avg("ecommerce.purchase_revenue_in_usd")
display(avg_state_purchases_df)

state,avg(ecommerce.purchase_revenue_in_usd AS purchase_revenue_in_usd)
SC,1036.6414360508602
AZ,1031.047341772152
LA,1044.6960085151673
MN,1043.4445019114005
NJ,1037.885416666666
DC,1026.901920236337
OR,1048.7469887955174
VA,1027.954827586206
RI,1043.0902803738318
KY,1062.6566689702831




And here the total quantity and sum of the purchase revenue for each combination of state and city.

In [0]:
city_purchase_quantities_df = df.groupBy("geo.state", "geo.city").sum("ecommerce.total_item_quantity", "ecommerce.purchase_revenue_in_usd")
display(city_purchase_quantities_df)

state,city,sum(ecommerce.total_item_quantity AS total_item_quantity),sum(ecommerce.purchase_revenue_in_usd AS purchase_revenue_in_usd)
MA,Attleboro,54,51166.8
IN,Indianapolis,1111,987157.4000000001
MN,North Branch,15,13684.5
IA,Ames,68,65635.8
MO,Kansas City,578,528030.7000000001
LA,Shreveport,241,217131.2
NH,Concord,48,46909.7
FL,Lake Wales,9,9010.0
CA,San Juan Capistrano,44,36872.6
WI,Lake Mills,4,4410.5




## Built-In Functions
In addition to DataFrame and Column transformation methods, there are a ton of helpful functions in Spark's built-in <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-functions-builtin.html" target="_blank">SQL functions</a> module.

In Scala, this is <a href="https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/functions$.html" target="_blank">**`org.apache.spark.sql.functions`**</a>, and <a href="https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions" target="_blank">**`pyspark.sql.functions`**</a> in Python. Functions from this module must be imported into your code.



### Aggregate Functions

Here are some of the built-in functions available for aggregation.

| Method | Description |
| --- | --- |
| approx_count_distinct | Returns the approximate number of distinct items in a group |
| avg | Returns the average of the values in a group |
| collect_list | Returns a list of objects with duplicates |
| corr | Returns the Pearson Correlation Coefficient for two columns |
| max | Compute the max value for each numeric columns for each group |
| mean | Compute the average value for each numeric columns for each group |
| stddev_samp | Returns the sample standard deviation of the expression in a group |
| sumDistinct | Returns the sum of distinct values in the expression |
| var_pop | Returns the population variance of the values in a group |

Use the grouped data method <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.GroupedData.agg.html#pyspark.sql.GroupedData.agg" target="_blank">**`agg`**</a> to apply built-in aggregate functions

This allows you to apply other transformations on the resulting columns, such as <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.alias.html" target="_blank">**`alias`**</a>.

In [0]:
from pyspark.sql.functions import sum

state_purchases_df = df.groupBy("geo.state").agg(sum("ecommerce.total_item_quantity").alias("total_purchases"))
display(state_purchases_df)

state,total_purchases
SC,1528
AZ,5886
LA,2145
MN,5098
NJ,2518
DC,790
OR,3317
VA,2973
RI,615
KY,1673





Apply multiple aggregate functions on grouped data

In [0]:
from pyspark.sql.functions import avg, approx_count_distinct

state_aggregates_df = (df
                       .groupBy("geo.state")
                       .agg(avg("ecommerce.total_item_quantity").alias("avg_quantity"),
                            approx_count_distinct("user_id").alias("distinct_users"))
                      )

display(state_aggregates_df)

state,avg_quantity,distinct_users
SC,1.1428571428571428,29155
AZ,1.1462512171372932,113246
LA,1.141564662054284,42411
MN,1.1463908252754667,103113
NJ,1.1403985507246377,49219
DC,1.1669128508124076,16239
OR,1.1614145658263306,65270
VA,1.139080459770115,54557
RI,1.1495327102803738,11982
KY,1.156185210780926,30511




### Math Functions
Here are some of the built-in functions for math operations.

| Method | Description |
| --- | --- |
| ceil | Computes the ceiling of the given column. |
| cos | Computes the cosine of the given value. |
| log | Computes the natural logarithm of the given value. |
| round | Returns the value of the column e rounded to 0 decimal places with HALF_UP round mode. |
| sqrt | Computes the square root of the specified float value. |

In [0]:
from pyspark.sql.functions import cos, sqrt

display(spark.range(10)  # Create a DataFrame with a single column called "id" with a range of integer values
        .withColumn("sqrt", sqrt("id"))
        .withColumn("cos", cos("id"))
       )

id,sqrt,cos
0,0.0,1.0
1,1.0,0.5403023058681398
2,1.4142135623730951,-0.4161468365471424
3,1.7320508075688772,-0.9899924966004454
4,2.0,-0.6536436208636119
5,2.23606797749979,0.28366218546322625
6,2.449489742783178,0.9601702866503661
7,2.6457513110645907,0.7539022543433046
8,2.8284271247461903,-0.14550003380861354
9,3.0,-0.9111302618846769




### Clean up classroom

In [0]:
DA.cleanup()

Resetting the learning environment:
| dropping the schema "muaazkhurshid_q1yg_da_asp"...(1 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/muaazkhurshid@uni-koblenz.de/apache-spark-programming-with-databricks"...(0 seconds)

Validating the locally installed datasets:
| listing local files...(3 seconds)
| validation completed...(3 seconds total)


&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>